<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Extension Activities</h2>	


TensorFlow (Optional)- Now, try using TensorFlow to categorize your images. The accuracy should be significantly higher due to the usage of nueral nets



In [38]:
import numpy as np
import tensorflow as tf
import pandas as pd
import sklearn
from sklearn import model_selection
from skimage.transform import resize
import skimage
from skimage import io
import tensorflow.contrib.layers as layers
from random import shuffle

In [39]:
NUM_ITER = 1000
IMG_SHAPE = (145, 145)
NUM_CLASSES = 20
LEARNING_RATE = .1
BATCH_SIZE = 10

In [40]:
def train_test_split(df):
    train_df, val_df = sklearn.model_selection.train_test_split(df, test_size=0.2)
    return train_df, val_df

#Split the data into a training set, and test set 
def accuracy(pred, actual):
    return np.mean(pred == actual)
# Calculate the accuracy percentage of the predicted values
def separate_features(df):
    feature_df = df.iloc[:,2:]
    labels_df = df.iloc[:,1]
    return feature_df, labels_df

In [41]:
training_data = pd.read_hdf('training_data.h5')

In [42]:
train, val = train_test_split(training_data)
# X_train, y_train = separate_features(train)
# X_train.reset_index().drop(columns=['index'])
# y_train.reset_index().drop(columns=['index'])
# y_train = y_train.astype('int')
# X_train_matrix = X_train.values
X_train_df = train['image']
y_train_df = train['label'].astype('int')
# X_train.head()

In [43]:
def preprocess(X):
    def preprocess_image(image):
        return resize(image / 255.0, IMG_SHAPE, mode='constant')
    images = []
    for image in X:
        images.append(preprocess_image(image))
    return np.array(images)
X_train = preprocess(X_train_df)
y_train = np.zeros((len(y_train_df), NUM_CLASSES))
y_train[np.arange(len(y_train_df)), y_train_df.values] = 1

In [44]:
def get_model():
    X_ph = tf.placeholder(tf.float32, shape=[None] + list(IMG_SHAPE) + [3])
    out = layers.convolution2d(X_ph, num_outputs=32, kernel_size=8, stride=4, activation_fn=tf.nn.relu)
    out = layers.convolution2d(out, num_outputs=64, kernel_size=4, stride=2, activation_fn=tf.nn.relu)
#     out = layers.convolution2d(out, num_outputs=64, kernel_size=3, stride=1, activation_fn=tf.nn.relu)

    out = layers.flatten(out)
    out = layers.fully_connected(out, num_outputs=512,         activation_fn=tf.nn.relu)
    y_hat = layers.fully_connected(out, num_outputs=NUM_CLASSES, activation_fn=None)
    return X_ph, y_hat

In [45]:
tf.reset_default_graph()
sess = tf.Session()

X_ph, y_hat = get_model()
# print(X_ph.get_shape(), X_train.shape)
# print(X_train.head(20))
y_ph = tf.placeholder(tf.float32, shape=[None, NUM_CLASSES])
loss = tf.losses.mean_squared_error(y_hat, y_ph)
train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
sess.run(tf.global_variables_initializer())

for i in range(NUM_ITER):
    X_batch = X_train[i:i+BATCH_SIZE]
    y_batch = y_train[i:i+BATCH_SIZE]
    
    sess.run(train_step, feed_dict={X_ph: X_batch, y_ph: y_batch})
    if i % 100 == 0:
        indices = np.arange(len(X_train))
        shuffle(indices)
        indices = indices[:200]
        curr_loss = sess.run(loss, feed_dict={X_ph: X_train[indices], y_ph: y_train[indices]})
        print('Loss: {}'.format(curr_loss))

Loss: 0.059891894459724426
Loss: 0.04894120618700981
Loss: 0.04989343136548996
Loss: 0.04744567722082138
Loss: 0.04868364706635475
Loss: 0.051085859537124634
Loss: 0.04690390080213547
Loss: 0.049169864505529404
Loss: 0.04333537817001343
Loss: 0.04249913617968559


In [46]:
X_val_df = val['image']
X_val = preprocess(X_val_df)
y_predicted = sess.run(y_hat, feed_dict={X_ph: X_val})
y_predicted = np.argmax(y_predicted, axis=1)
accuracy(y_predicted, val['label'].astype('int'))

0.33222591362126247